In [1]:
import json
import gensim
from gensim.models.fasttext import FastText
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import sent_tokenize
from nltk import WordPunctTokenizer
import pandas as pd
import re
import numpy as np

ModuleNotFoundError: No module named 'gensim'

In [ ]:
with open ("Lyrics_analysis-master\Lyrics_analysis-master\songs.json", 'r') as f:
    songs = json.load(f)

In [ ]:
'''def max_len(lyrics):
    max_len = 0
    for i in range(len(lyrics)):
        print(lyrics)
        if len(lyrics[i]['lyrics']) > max_len:
            max_len = len(lyrics[i]['lyrics'])
            
    return max_len'''

print(len(songs))
print(songs[0])
limit = len(songs)

bad = []

max_len = 0
song = ""

for i in range(limit):
    try:
        if len(songs[i]['lyrics']) > max_len:
            max_len = len(songs[i]['lyrics'])
            song = songs[i]
    except TypeError:
        bad.append(i)
        max_len = max_len
    except KeyError:
        bad.append(i)
        max_len = max_len

In [ ]:
titles = []
lyrics = []
bad2 = []

for i in range(limit):
    try:
        titles.append(songs[i]['title'])
        lyrics.append(songs[i]['lyrics'])
    except TypeError:
        bad2.append(i)
    except KeyError:
        bad2.append(i)
        max_len = max_len
        
assert(bad2 == bad)

df = pd.DataFrame({'songs' : titles, 'lyrics' : lyrics})
df.head()

In [ ]:
stemmer = WordNetLemmatizer()
en_stop = set(nltk.corpus.stopwords.words('english'))

def preprocess(line):  
    
    line = re.sub(r'\W', ' ', str(line))
    line = re.sub(r'\s+[a-zA-Z]\s+', ' ', line)
    line = re.sub(r'\^[a-zA-Z]\s+', ' ', line)
    line = re.sub(r'\s+', ' ', line, flags=re.I)
    
    line = line.lower()
    
    tokens = line.split()    
    tokens = [stemmer.lemmatize(word) for word in tokens]
    tokens = [word for word in tokens if word not in en_stop]
    #tokens = [word for word in tokens if len(word) > 3]
    
    return tokens
    
def preprocess_all(lyrics):
    ret = []
    for line in lyrics:
        tokens = preprocess(line)
        if not tokens == []:
            ret.append(tokens)
            
    return ret

In [ ]:
df['preprocessed_lyrics'] = df['lyrics'].apply(preprocess_all)
df.head()

In [ ]:
max_len = 0
for index, row in df.iterrows():
    lyrics = row['preprocessed_lyrics']
    if len(lyrics) > max_len:
        max_len = len(lyrics)
        
print(max_len)

In [ ]:
embedding_size = 100
window_size = 40
min_word = 5
down_sampling = 1e-2

'''ft_model = FastText(size=embedding_size,
                      window=window_size,
                      min_count=min_word,
                      sample=down_sampling,
                      sg=0,
                      iter=100)'''
model = gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin/GoogleNews-vectors-negative300.bin', binary=True)  

In [ ]:
lyrics_embeddings = np.zeros((1, 300))

def line2vec(line):
    line_embedding = np.zeros(300)
    exceptions = 0
    for word in line:
        #print(model[word].shape)
        try:
            line_embedding += model[word]
        except KeyError:
            #print(word)
            exceptions += 1
    
    return line_embedding / (len(line) - exceptions)

for index, row in df.iterrows():
    lyrics = row['preprocessed_lyrics']
    
    if index % 500 == 0:
        print(index)
    
    lyric_embedding = np.zeros(300)
    for line in lyrics:
        lyric_embedding += line2vec(line)
        
    lyric_embedding /= len(lyrics)
    
    lyric_embedding = np.reshape(lyric_embedding, (1, -1))
    #print(lyrics_embeddings.shape)
    #print(lyric_embedding.shape)
    lyrics_embeddings = np.concatenate((lyrics_embeddings, lyric_embedding), axis=0)
    
print(len(lyrics))
lyrics_embeddings = np.delete(lyrics_embeddings, [0], axis=0)

In [ ]:
np.save('embeddings.npy', lyrics_embeddings)
lyrics_embeddings

In [ ]:
test = np.load('embeddings.npy')
test